In [1]:
import pandas as pd

In [2]:
# directory = '/media/share/DataSaves/wikitable_extract_220814_test'
directory = '/Volumes/14TB/DataSaves/wikitable_extract_220814_test'

In [3]:
import glob
test = glob.glob(directory + '/*.parquet')

In [4]:
len(test)

265

In [5]:
# ! pip inst?all py-wikimarkup

In [6]:
from wikimarkup.parser import Parser
from bs4 import BeautifulSoup

parser = Parser()

def get_tables(markdowntext):
    output = []
    try:
        html = parser.parse(markdowntext)
        soup = BeautifulSoup(html, 'html.parser')
        tables = soup.find_all('table')
        df_tables = pd.read_html(html)
        if len(tables) != len(df_tables):
            print("tables offset..")
            return []
        output = []
        for i, table in enumerate(tables):
            # all previous headings in order
            headings = []
            for k in range(6):
                h = table.find_previous_sibling(f'h{k}')
                headings.append(
                    h.text if h else ''
                )
            output.append((i, "\n".join(headings), df_tables[i]))
    except Exception as e:
        print("Failed to parse...")
    return output

def get_all_tables(path):
    data = pd.read_parquet(path)
    data = data['page_text'].groupby(data['page_title_text']).first()
    for title, text in data.iteritems():
        result = get_tables(text)
        for row in result:
            yield title, *row

In [7]:
from sketch import Portfolio

In [8]:
import time
st = time.time()
results = []
for k, thing in enumerate(test[0:50]):
    print(k, thing)
    for i, x in enumerate(get_all_tables(thing)):
        results.append(x)
        if i%100 == 0:
            print(i, time.time() - st)
            pf = Portfolio()
            for result in results:
                pf.add_wikitable(result[0], result[1], result[2], result[3])
            print(f"About to upload... {len(pf.sketchpads)}")
            pf.upload(apiKey="8f79be2b6d0d47ccb8192e46f38c80ce")
            results = []
            

0 /Volumes/14TB/DataSaves/wikitable_extract_220814_test/11771.parquet
0 26.573625802993774
About to upload... 2
Failed to parse...
Failed to parse...
Failed to parse...
100 29.744261980056763
About to upload... 524
200 37.14842700958252
About to upload... 514
Failed to parse...
300 44.4172739982605
About to upload... 616
Failed to parse...
Failed to parse...
Failed to parse...
400 52.819101095199585
About to upload... 487
Failed to parse...
500 61.81754493713379
About to upload... 521
Failed to parse...
Failed to parse...
Failed to parse...
600 68.66260313987732
About to upload... 547
tables offset..
Failed to parse...
700 76.20478296279907
About to upload... 460
